This notebook is based on my previous notebook [Baseline_For_Beginners](https://www.kaggle.com/code/adamlogman/baseline-for-beginners),and [Dmitry Uarov](https://www.kaggle.com/dmitryuarov) 's [NeurIPS | Baseline + External data](https://www.kaggle.com/code/dmitryuarov/neurips-baseline-external-data) notebook with modifications to the model

# Import Dependencies 

In [1]:
!pip install /kaggle/input/rdkit-2025-3-3-cp311/rdkit-2025.3.3-cp311-cp311-manylinux_2_28_x86_64.whl

Processing /kaggle/input/rdkit-2025-3-3-cp311/rdkit-2025.3.3-cp311-cp311-manylinux_2_28_x86_64.whl


In [2]:
 # Importing Required Libraries\nLet's begin by importing the essential Python libraries needed for data processing, visualization, and modeling.

import pandas as pd
import numpy as np


from sklearn.ensemble import HistGradientBoostingRegressor,ExtraTreesRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error


import networkx as nx
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors
from rdkit.Chem import rdmolops
from rdkit import Chem

import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

In [3]:
class CFG:
    TARGETS = ['Tg', 'FFV', 'Tc', 'Density', 'Rg']
    SEED = 42
    FOLDS = 10

In [4]:
useless_cols = [   
    
    'MaxPartialCharge', 
    # Nan data
    'BCUT2D_MWHI',
    'BCUT2D_MWLOW',
    'BCUT2D_CHGHI',
    'BCUT2D_CHGLO',
    'BCUT2D_LOGPHI',
    'BCUT2D_LOGPLOW',
    'BCUT2D_MRHI',
    'BCUT2D_MRLOW',

    # Constant data
    'NumRadicalElectrons',
    'SMR_VSA8',
    'SlogP_VSA9',
    'fr_barbitur',
    'fr_benzodiazepine',
    'fr_dihydropyridine',
    'fr_epoxide',
    'fr_isothiocyan',
    'fr_lactam',
    'fr_nitroso',
    'fr_prisulfonamd',
    'fr_thiocyan',

    # High correlated data >0.95
    'MaxEStateIndex',
    'HeavyAtomMolWt',
    'ExactMolWt',
    'NumValenceElectrons',
    'Chi0',
    'Chi0n',
    'Chi0v',
    'Chi1',
    'Chi1n',
    'Chi1v',
    'Chi2n',
    'Kappa1',
    'LabuteASA',
    'HeavyAtomCount',
    'MolMR',
    'Chi3n',
    'BertzCT',
    'Chi2v',
    'Chi4n',
    'HallKierAlpha',
    'Chi3v',
    'Chi4v',
    'MinAbsPartialCharge',
    'MinPartialCharge',
    'MaxAbsPartialCharge',
    'FpDensityMorgan2',
    'FpDensityMorgan3',
    'Phi',
    'Kappa3',
    'fr_nitrile',
    'SlogP_VSA6',
    'NumAromaticCarbocycles',
    'NumAromaticRings',
    'fr_benzene',
    'VSA_EState6',
    'NOCount',
    'fr_C_O',
    'fr_C_O_noCOO',
    'NumHDonors',
    'fr_amide',
    'fr_Nhpyrrole',
    'fr_phenol',
    'fr_phenol_noOrthoHbond',
    'fr_COO2',
    'fr_halogen',
    'fr_diazo',
    'fr_nitro_arom',
    'fr_phos_ester'
]

SEED = 42

# Read Files

### Main Files

In [5]:
 #We will load both the training and test datasets using pandas, and store test IDs 
train=pd.read_csv('/kaggle/input/neurips-open-polymer-prediction-2025/train.csv')
test=pd.read_csv('/kaggle/input/neurips-open-polymer-prediction-2025/test.csv')
ss=pd.read_csv('/kaggle/input/neurips-open-polymer-prediction-2025/sample_submission.csv')
ID=test['id'].copy()

In [6]:
train.head()

,id,SMILES,Tg,FFV,Tc,Density,Rg
0,87817,*CC(*)c1ccccc1C(=O)OCCCCCC,NaN,0.374645,0.205667,NaN,NaN
1,106919,*Nc1ccc([C@H](CCC)c2ccc(C3(c4ccc([C@@H](CCC)c5...,NaN,0.370410,NaN,NaN,NaN
2,388772,*Oc1ccc(S(=O)(=O)c2ccc(Oc3ccc(C4(c5ccc(Oc6ccc(...,NaN,0.378860,NaN,NaN,NaN
3,519416,*Nc1ccc(-c2c(-c3ccc(C)cc3)c(-c3ccc(C)cc3)c(N*)...,NaN,0.387324,NaN,NaN,NaN
4,539187,*Oc1ccc(OC(=O)c2cc(OCCCCCCCCCOCC3CCCN3c3ccc([N...,NaN,0.355470,NaN,NaN,NaN


In [7]:
train.shape, test.shape

((7973, 7), (3, 2))

In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7973 entries, 0 to 7972
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id       7973 non-null   int64  
 1   SMILES   7973 non-null   object 
 2   Tg       511 non-null    float64
 3   FFV      7030 non-null   float64
 4   Tc       737 non-null    float64
 5   Density  613 non-null    float64
 6   Rg       614 non-null    float64
dtypes: float64(5), int64(1), object(1)
memory usage: 436.2+ KB


### Extra Files

In [9]:
tc_smiles =pd.read_csv('/kaggle/input/tc-smiles/Tc_SMILES.csv')
tg_smiles =pd.read_csv('/kaggle/input/smiles-extra-data/JCIM_sup_bigsmiles.csv')
ktg_smiles =pd.read_excel('/kaggle/input/smiles-extra-data/data_tg3.xlsx')
de_smiles =pd.read_excel('/kaggle/input/smiles-extra-data/data_dnst1.xlsx')

# Preprocessing 

In [10]:
def make_smile_canonical(smile): # To avoid duplicates, for example: canonical '*C=C(*)C' == '*C(=C*)C'
    try:
        mol = Chem.MolFromSmiles(smile)
        canon_smile = Chem.MolToSmiles(mol, canonical=True)
        return canon_smile
    except:
        return np.nan

train['SMILES'] = train['SMILES'].apply(lambda s: make_smile_canonical(s))
test['SMILES'] = test['SMILES'].apply(lambda s: make_smile_canonical(s))

In [11]:
ktg_smiles.rename(columns={'Tg [K]': 'Tg'}, inplace=True)
tg_smiles.rename(columns={'Tg (C)': 'Tg'}, inplace=True)
tc_smiles.rename(columns={'TC_mean': 'Tc'}, inplace=True)
de_smiles.rename(columns={'density(g/cm3)': 'Density'}, inplace=True)

In [12]:
de_smiles['SMILES'] = de_smiles['SMILES'].apply(lambda s: make_smile_canonical(s))
de_smiles = de_smiles[(de_smiles['SMILES'].notnull())&(de_smiles['Density'].notnull())&(de_smiles['Density'] != 'nylon')]
de_smiles['Density'] = de_smiles['Density'].astype('float64')
de_smiles['Density'] -= 0.118

ktg_smiles['Tg'] = ktg_smiles['Tg'] - 273.15

[13:59:24] SMILES Parse Error: syntax error while parsing: *O[Si](*)([R])[R]
[13:59:24] SMILES Parse Error: check for mistakes around position 12:
[13:59:24] *O[Si](*)([R])[R]
[13:59:24] ~~~~~~~~~~~^
[13:59:24] SMILES Parse Error: Failed parsing SMILES '*O[Si](*)([R])[R]' for input: '*O[Si](*)([R])[R]'
[13:59:24] SMILES Parse Error: syntax error while parsing: *NC(=O)c4ccc3c(=O)n(c2ccc([R]c1ccc(*)cc1)cc2)c(=O)c3c4
[13:59:24] SMILES Parse Error: check for mistakes around position 28:
[13:59:24] c4ccc3c(=O)n(c2ccc([R]c1ccc(*)cc1)cc2)c(=
[13:59:24] ~~~~~~~~~~~~~~~~~~~~^
[13:59:24] SMILES Parse Error: Failed parsing SMILES '*NC(=O)c4ccc3c(=O)n(c2ccc([R]c1ccc(*)cc1)cc2)c(=O)c3c4' for input: '*NC(=O)c4ccc3c(=O)n(c2ccc([R]c1ccc(*)cc1)cc2)c(=O)c3c4'
[13:59:24] SMILES Parse Error: syntax error while parsing: O=C=N[R1]N=C=O.O[R2]O.O[R3]O
[13:59:24] SMILES Parse Error: check for mistakes around position 7:
[13:59:24] O=C=N[R1]N=C=O.O[R2]O.O[R3]O
[13:59:24] ~~~~~~^
[13:59:24] SMILES Parse Error: F

In [13]:
def preprocessing(df):
    desc_names = [desc[0] for desc in Descriptors.descList if desc[0] not in useless_cols]
    descriptors = [compute_all_descriptors(smi) for smi in df['SMILES'].to_list()]

    graph_feats = {'graph_diameter': [], 'avg_shortest_path': [], 'num_cycles': []}
    for smile in df['SMILES']:
         compute_graph_features(smile, graph_feats)
        
    result = pd.concat(
        [
            pd.DataFrame(descriptors, columns=desc_names),
            pd.DataFrame(graph_feats)
        ],
        axis=1
    )

    result = result.replace([-np.inf, np.inf], np.nan)
    return result

# Feature Extraction 

In [14]:
# SMILESを正規化（canonical化）
# 重複する SMILES を集約して平均 target 値に
# df_train に存在するが欠損している target に、df_extra から値を補完

def add_extra_data(df_train, df_extra, target):
    n_samples_before = len(df_train[df_train[target].notnull()])
    
    df_extra['SMILES'] = df_extra['SMILES'].apply(lambda s: make_smile_canonical(s))
    df_extra = df_extra.groupby('SMILES', as_index=False)[target].mean()
    cross_smiles = set(df_extra['SMILES']) & set(df_train['SMILES'])
    unique_smiles_extra = set(df_extra['SMILES']) - set(df_train['SMILES'])

    # Make priority target value from competition's df
    for smile in df_train[df_train[target].notnull()]['SMILES'].tolist():
        if smile in cross_smiles:
            cross_smiles.remove(smile)

    # Imput missing values for competition's SMILES
    for smile in cross_smiles:
        df_train.loc[df_train['SMILES']==smile, target] = df_extra[df_extra['SMILES']==smile][target].values[0]
    
    df_train = pd.concat([df_train, df_extra[df_extra['SMILES'].isin(unique_smiles_extra)]], axis=0).reset_index(drop=True)

    n_samples_after = len(df_train[df_train[target].notnull()])
    print(f'\nFor target "{target}" added {n_samples_after-n_samples_before} new samples!')
    print(f'New unique SMILES: {len(unique_smiles_extra)}')
    return df_train

train = add_extra_data(train, tc_smiles, 'Tc')
train = add_extra_data(train, tg_smiles, 'Tg')
train = add_extra_data(train, ktg_smiles, 'Tg')
train = add_extra_data(train, de_smiles, 'Density')


For target "Tc" added 129 new samples!
New unique SMILES: 129

For target "Tg" added 151 new samples!
New unique SMILES: 136

For target "Tg" added 499 new samples!
New unique SMILES: 499

For target "Density" added 634 new samples!
New unique SMILES: 524


In [15]:
def compute_all_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return [None] * len(desc_names)
    return [desc[1](mol) for desc in Descriptors.descList if desc[0] not in useless_cols]

def compute_graph_features(smiles, graph_feats):
    mol = Chem.MolFromSmiles(smiles)
    adj = rdmolops.GetAdjacencyMatrix(mol)
    G = nx.from_numpy_array(adj)

    graph_feats['graph_diameter'].append(nx.diameter(G) if nx.is_connected(G) else 0)
    graph_feats['avg_shortest_path'].append(nx.average_shortest_path_length(G) if nx.is_connected(G) else 0)
    graph_feats['num_cycles'].append(len(list(nx.cycle_basis(G))))

train = pd.concat([train, preprocessing(train)], axis=1)
test = pd.concat([test, preprocessing(test)], axis=1)

# Find constant columns for each target
all_features = train.columns[7:].tolist()
features = {}
for target in CFG.TARGETS:
    const_descs = []
    for col in train.columns.drop(CFG.TARGETS):
        if train[train[target].notnull()][col].nunique() == 1:
            const_descs.append(col)
    features[target] = [f for f in all_features if f not in const_descs]

print(train.shape)
train['Ipc']=np.log10(train['Ipc'])  
for n in train.columns[7:]:
    train[n]=train[n].replace(-np.inf,np.nan)
    train[n]=train[n].replace(np.inf,np.nan)    
    train[n].fillna(train[n].mean())
  
print(train.shape)
test['Ipc']=np.log10(test['Ipc'])
for n in test.columns[7:]:
    train[n]=train[n].replace(-np.inf,np.nan)
    train[n]=train[n].replace(np.inf,np.nan)      
    test[n].fillna(train[n].mean())

(9261, 158)
(9261, 158)


In [16]:
train

,id,SMILES,Tg,FFV,Tc,Density,Rg,MaxAbsEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MolWt,FpDensityMorgan1,AvgIpc,BalabanJ,Ipc,Kappa2,PEOE_VSA1,PEOE_VSA10,PEOE_VSA11,PEOE_VSA12,PEOE_VSA13,PEOE_VSA14,PEOE_VSA2,PEOE_VSA3,PEOE_VSA4,PEOE_VSA5,PEOE_VSA6,PEOE_VSA7,PEOE_VSA8,PEOE_VSA9,SMR_VSA1,SMR_VSA10,SMR_VSA2,SMR_VSA3,SMR_VSA4,SMR_VSA5,SMR_VSA6,SMR_VSA7,SMR_VSA9,SlogP_VSA1,SlogP_VSA10,SlogP_VSA11,SlogP_VSA12,SlogP_VSA2,SlogP_VSA3,SlogP_VSA4,SlogP_VSA5,SlogP_VSA7,SlogP_VSA8,TPSA,EState_VSA1,EState_VSA10,EState_VSA11,EState_VSA2,EState_VSA3,EState_VSA4,EState_VSA5,EState_VSA6,EState_VSA7,EState_VSA8,EState_VSA9,VSA_EState1,VSA_EState10,VSA_EState2,VSA_EState3,VSA_EState4,VSA_EState5,VSA_EState7,VSA_EState8,VSA_EState9,FractionCSP3,NHOHCount,NumAliphaticCarbocycles,NumAliphaticHeterocycles,NumAliphaticRings,NumAmideBonds,NumAromaticHeterocycles,NumAtomStereoCenters,NumBridgeheadAtoms,NumHAcceptors,NumHeteroatoms,NumHeterocycles,NumRotatableBonds,NumSaturatedCarbocycles,NumSaturatedHeterocycles,NumSaturatedRings,NumSpiroAtoms,NumUnspecifiedAtomStereoCenters,RingCount,MolLogP,fr_Al_COO,fr_Al_OH,fr_Al_OH_noTert,fr_ArN,fr_Ar_COO,fr_Ar_N,fr_Ar_NH,fr_Ar_OH,fr_COO,fr_C_S,fr_HOCCN,fr_Imine,fr_NH0,fr_NH1,fr_NH2,fr_N_O,fr_Ndealkylation1,fr_Ndealkylation2,fr_SH,fr_aldehyde,fr_alkyl_carbamate,fr_alkyl_halide,fr_allylic_oxid,fr_amidine,fr_aniline,fr_aryl_methyl,fr_azide,fr_azo,fr_bicyclic,fr_ester,fr_ether,fr_furan,fr_guanido,fr_hdrzine,fr_hdrzone,fr_imidazole,fr_imide,fr_isocyan,fr_ketone,fr_ketone_Topliss,fr_lactone,fr_methoxy,fr_morpholine,fr_nitro,fr_nitro_arom_nonortho,fr_oxazole,fr_oxime,fr_para_hydroxylation,fr_phos_acid,fr_piperdine,fr_piperzine,fr_priamide,fr_pyridine,fr_quatN,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiophene,fr_unbrch_alkane,fr_urea,graph_diameter,avg_shortest_path,num_cycles
0,87817.0,*CC(*)c1ccccc1C(=O)OCCCCCC,NaN,0.374645,0.205667,0.932,NaN,12.144536,0.105927,-0.105927,0.500278,13.705882,232.323,1.411765,2.456411,2.563477,4.261290,5.852071,0.000000,0.000000,0.00000,0.000000,0.000000,103.451541,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9.531400,5.969305,0.0,0.000000,0.000000,45.951583,6.606882,35.392371,0.000000,0.000000,0.000000,0.000000,0.000000,12.576187,4.736863,0.000000,54.949285,6.923737,0.000000,26.30,0.000000,4.794537,0.0,5.969305,6.606882,24.825916,25.328832,0.000000,12.132734,19.056471,4.736863,6.120445,0.000000,12.144536,0.000000,2.105648,-0.105927,5.344954,4.148954,0.000000,0.533333,0,0,0,0,0,0,1,0,2,4,0,8,0,0,0,0,1,1,3.98170,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,12,4.736842,1
1,106919.0,*Nc1ccc([C@H](CCC)c2ccc(C3(c4ccc([C@@H](CCC)c5...,NaN,0.370410,NaN,NaN,NaN,3.523412,0.098918,0.098918,0.125364,16.777778,598.919,0.577778,3.135512,1.451540,10.636376,14.715582,0.000000,0.000000,0.00000,0.000000,0.000000,222.907896,0.000000,0.000000,0.000000,0.000000,32.607024,18.759549,0.000000,0.000000,0.000000,11.374773,0.0,0.000000,5.917906,115.071874,11.467335,130.442582,0.000000,11.467335,11.374773,0.000000,0.000000,0.000000,5.414990,5.917906,143.037592,0.000000,0.000000,24.06,0.000000,0.000000,0.0,5.414990,11.835812,5.917906,121.805341,0.000000,0.000000,129.300420,0.000000,0.326442,0.000000,0.000000,0.000000,18.579143,1.794202,15.456662,6.971589,0.000000,0.441860,2,1,0,1,0,0,2,0,2,4,0,16,1,0,1,0,0,5,12.35960,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,22,8.364477,5
2,388772.0,*Oc1ccc(S(=O)(=O)c2ccc(Oc3ccc(C4(c5ccc(Oc6ccc(...,NaN,0.378860,NaN,NaN,NaN,13.714745,0.107441,-3.829434,0.092387,16.301370,1003.207,0.397260,3.440031,0.743739,16.833145,19.899082,9.473726,22.998047,0.00000,9.837253,0.000000,222.531582,0.000000,8.417797,0.000000,0.000000,43.527933,96.764951,5.414990,9.790967,40.947247,43.676159,0.0,0.000000,0.000000,76.363497,0.000000,227.523761,40.246583,25.383483,0.00

# Data Preparation For Model Training 

In [17]:
# We'll separate train to be one model for each target variable.
t_1=train[['SMILES','Tg']].copy()
t_2=train[['SMILES','FFV']].copy()
t_3=train[['SMILES','Tc']].copy()
t_4=train[['SMILES','Density']].copy()
t_5=train[['SMILES','Rg']].copy()

# We will drop the rows with missing values related to that target after separation.
#This is important , dropping them beforehand would result Null for all data.
t_1.dropna(inplace=True)
t_2.dropna(inplace=True)
t_3.dropna(inplace=True)
t_4.dropna(inplace=True)
t_5.dropna(inplace=True)

In [18]:
train=train.drop(['id','Tg','FFV','Tc','Density','Rg'],axis=1)
test=test.drop(['id','SMILES'],axis=1)

In [19]:
tg=t_1.merge(train,on='SMILES',how='left')
ffv=t_2.merge(train,on='SMILES',how='left')
tc=t_3.merge(train,on='SMILES',how='left')
density=t_4.merge(train,on='SMILES',how='left')
rg=t_5.merge(train,on='SMILES',how='left')

In [20]:
for i in (tg,tc,density,ffv,rg):
    i.drop('SMILES',axis=1,inplace=True)
    i.dropna(inplace=True)

In [21]:
tg.head()

,Tg,MaxAbsEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MolWt,FpDensityMorgan1,AvgIpc,BalabanJ,Ipc,Kappa2,PEOE_VSA1,PEOE_VSA10,PEOE_VSA11,PEOE_VSA12,PEOE_VSA13,PEOE_VSA14,PEOE_VSA2,PEOE_VSA3,PEOE_VSA4,PEOE_VSA5,PEOE_VSA6,PEOE_VSA7,PEOE_VSA8,PEOE_VSA9,SMR_VSA1,SMR_VSA10,SMR_VSA2,SMR_VSA3,SMR_VSA4,SMR_VSA5,SMR_VSA6,SMR_VSA7,SMR_VSA9,SlogP_VSA1,SlogP_VSA10,SlogP_VSA11,SlogP_VSA12,SlogP_VSA2,SlogP_VSA3,SlogP_VSA4,SlogP_VSA5,SlogP_VSA7,SlogP_VSA8,TPSA,EState_VSA1,EState_VSA10,EState_VSA11,EState_VSA2,EState_VSA3,EState_VSA4,EState_VSA5,EState_VSA6,EState_VSA7,EState_VSA8,EState_VSA9,VSA_EState1,VSA_EState10,VSA_EState2,VSA_EState3,VSA_EState4,VSA_EState5,VSA_EState7,VSA_EState8,VSA_EState9,FractionCSP3,NHOHCount,NumAliphaticCarbocycles,NumAliphaticHeterocycles,NumAliphaticRings,NumAmideBonds,NumAromaticHeterocycles,NumAtomStereoCenters,NumBridgeheadAtoms,NumHAcceptors,NumHeteroatoms,NumHeterocycles,NumRotatableBonds,NumSaturatedCarbocycles,NumSaturatedHeterocycles,NumSaturatedRings,NumSpiroAtoms,NumUnspecifiedAtomStereoCenters,RingCount,MolLogP,fr_Al_COO,fr_Al_OH,fr_Al_OH_noTert,fr_ArN,fr_Ar_COO,fr_Ar_N,fr_Ar_NH,fr_Ar_OH,fr_COO,fr_C_S,fr_HOCCN,fr_Imine,fr_NH0,fr_NH1,fr_NH2,fr_N_O,fr_Ndealkylation1,fr_Ndealkylation2,fr_SH,fr_aldehyde,fr_alkyl_carbamate,fr_alkyl_halide,fr_allylic_oxid,fr_amidine,fr_aniline,fr_aryl_methyl,fr_azide,fr_azo,fr_bicyclic,fr_ester,fr_ether,fr_furan,fr_guanido,fr_hdrzine,fr_hdrzone,fr_imidazole,fr_imide,fr_isocyan,fr_ketone,fr_ketone_Topliss,fr_lactone,fr_methoxy,fr_morpholine,fr_nitro,fr_nitro_arom_nonortho,fr_oxazole,fr_oxime,fr_para_hydroxylation,fr_phos_acid,fr_piperdine,fr_piperzine,fr_priamide,fr_pyridine,fr_quatN,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiophene,fr_unbrch_alkane,fr_urea,graph_diameter,avg_shortest_path,num_cycles
0,208.639749,11.248449,0.023312,-0.334079,0.435492,13.615385,185.183,1.384615,2.210448,3.709964,3.151338,3.993205,0.0,0.0,0.0,0.0,0.0,74.906467,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,14.383612,18.100520,0.0,16.367245,0.0,12.965578,13.089513,0.000000,0.000000,16.367245,0.0,0.000000,0.000000,37.231874,14.383612,0.0,6.923737,0.000000,0.000000,87.30,0.0,14.383612,0.0,37.231874,0.000000,0.000000,0.000000,6.923737,0.000000,16.367245,0.000000,-0.357391,0.000000,33.004449,4.941399,3.000789,-0.488941,0.133398,1.818204,0.000000,0.571429,3,0,0,0,2,0,1,0,4,8,0,6,0,0,0,0,1,0,-2.2629,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10,4.285714,0
1,-41.266724,2.142025,0.580086,0.580086,0.318231,12.571429,268.538,0.714286,2.306121,2.828262,3.566187,10.902614,0.0,0.0,0.0,0.0,0.0,106.749724,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,44.042496,0.0,0.000000,0.0,45.448667,17.258561,0.000000,0.000000,0.000000,0.0,0.000000,44.042496,17.258561,0.000000,0.0,38.524930,6.923737,0.000000,0.00,0.0,0.000000,0.0,0.000000,0.000000,0.000000,55.783491,6.923737,0.000000,44.042496,0.000000,1.277360,2.142025,0.000000,0.000000,0.000000,4.148997,8.837643,1.611538,6.315771,1.000000,0,0,0,0,0,0,0,0,4,6,0,13,0,0,0,0,0,0,5.5217,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,15,5.666667,0
2,-17.282022,2.408148,0.540428,0.540428,0.448030,13.600000,138.254,1.000000,2.098391,2.813864,2.689242,5.519191,0.0,0.0,0.0,0.0,0.0,64.524444,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,51.869489,0.000000,12.654956,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,44.945751,6.923737,0.000000,0.00,0.0,0.000000,0.0,0.000000,0.000000,0.000000,38.524930,13.344559,0.000000,12.654956,0.000000,1.252280,0.000000,0.000000,0.000000,0.000000,0.000000,12.721612,4.026108,0.000000,0.800000,0,0,0,0,0,0,0,0,0,2,0,8,0,0,0,0,0,0,3.7477,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

# Model 

In [22]:
# Let’s define a reusable function to train and evaluate our machine learning model.

def model(train_d,test_d,model,target,submission=False):
    # We divide the data into training and validation sets for model evaluation
    X=train_d.drop(target,axis=1)
    y=train_d[target].copy()
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=10)

    Model=model()
    if submission==False:
       Model.fit(X_train,y_train)
       y_pred=Model.predict(X_test)
       return mean_absolute_error(y_pred,y_test)         # We assess our model performance using MAE metric
    if submission==True:
       Model.fit(X,y)
       submission=Model.predict(test_d)
       return submission
        

# **Model Evaluation

In [23]:
from sklearn.ensemble import (
    RandomForestRegressor,
    ExtraTreesRegressor,
    GradientBoostingRegressor,
    HistGradientBoostingRegressor
)
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

In [24]:
from sklearn.preprocessing import StandardScaler

models_to_compare = {
    'RandomForest': lambda: RandomForestRegressor(
        n_estimators=500,
        max_depth=12,
        min_samples_leaf=2,
        max_features='sqrt',
        n_jobs=-1,
        random_state=SEED
    ),
    'ExtraTrees': lambda: ExtraTreesRegressor(
        n_estimators=500,
        max_depth=12,
        min_samples_leaf=2,
        max_features='sqrt',
        n_jobs=-1,
        random_state=SEED
    ),
    'GradientBoosting': lambda: GradientBoostingRegressor(
        n_estimators=1000,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.8,
        max_features='sqrt',
        random_state=SEED
    ),
    'HistGradientBoosting': lambda: HistGradientBoostingRegressor(
        max_iter=1000,
        learning_rate=0.05,
        max_leaf_nodes=31,
        early_stopping='auto',
        random_state=SEED
    ),
    'XGBoost': lambda: XGBRegressor(
        n_estimators=1000,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        tree_method='auto',   # change to 'gpu_hist' if GPU-compatible wheel and GPU available
        random_state=SEED,
        verbosity=0,
        n_jobs=-1
    ),
    'LightGBM': lambda: LGBMRegressor(
        n_estimators=1000,
        learning_rate=0.05,
        num_leaves=64,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=SEED,
        n_jobs=-1,
        verbose=-1
    ),
    'CatBoost': lambda: CatBoostRegressor(
        iterations=1000,
        learning_rate=0.05,
        depth=6,
        early_stopping_rounds=50,
        random_seed=SEED,
        verbose=0
    ),
}

scaler = StandardScaler()
tg["Tg"] = scaler.fit_transform(np.array(tg["Tg"]).reshape(-1, 1))

for name, model_init in models_to_compare.items():
    try:
        score = model(tg, test, model_init, 'Tg', submission=False)
        print(f'{name:<20}: MAE = {score:.5f}')
    except Exception as e:
        print(f'{name:<20}: Failed with error: {e}')

RandomForest        : MAE = 0.30734
ExtraTrees          : MAE = 0.32888
GradientBoosting    : MAE = 0.28657
HistGradientBoosting: MAE = 0.30506
XGBoost             : MAE = 0.28546
LightGBM            : MAE = 0.29355
CatBoost            : MAE = 0.28643


In [25]:
models_to_compare = {
    'RandomForest': lambda: RandomForestRegressor(
        n_estimators=500,
        max_depth=12,
        min_samples_leaf=2,
        max_features='sqrt',
        n_jobs=-1,
        random_state=SEED
    ),
    'ExtraTrees': lambda: ExtraTreesRegressor(
        n_estimators=500,
        max_depth=12,
        min_samples_leaf=2,
        max_features='sqrt',
        n_jobs=-1,
        random_state=SEED
    ),
    'GradientBoosting': lambda: GradientBoostingRegressor(
        n_estimators=1000,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.8,
        max_features='sqrt',
        random_state=SEED
    ),
    'HistGradientBoosting': lambda: HistGradientBoostingRegressor(
        max_iter=1000,
        learning_rate=0.05,
        max_leaf_nodes=31,
        early_stopping='auto',
        random_state=SEED
    ),
    'XGBoost': lambda: XGBRegressor(
        n_estimators=1000,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        tree_method='auto',   # change to 'gpu_hist' if GPU-compatible wheel and GPU available
        random_state=SEED,
        verbosity=0,
        n_jobs=-1
    ),
    'LightGBM': lambda: LGBMRegressor(
        n_estimators=1000,
        learning_rate=0.05,
        num_leaves=64,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=SEED,
        n_jobs=-1,
        verbose=-1
    ),
    'CatBoost': lambda: CatBoostRegressor(
        iterations=1000,
        learning_rate=0.05,
        depth=6,
        early_stopping_rounds=50,
        random_seed=SEED,
        verbose=0
    ),
}

for name, model_cls in models_to_compare.items():
    try:
        score = model(ffv, test, model_cls, 'FFV', submission=False)
        print(f'{name:<20}: MAE = {score:.5f}')
    except Exception as e:
        print(f'{name:<20}: Failed with error: {e}')

RandomForest        : MAE = 0.00906
ExtraTrees          : MAE = 0.01139
GradientBoosting    : MAE = 0.00645
HistGradientBoosting: MAE = 0.00704
XGBoost             : MAE = 0.00650
LightGBM            : MAE = 0.00701
CatBoost            : MAE = 0.00693


In [26]:
models_to_compare = {
    'RandomForest': lambda: RandomForestRegressor(
        n_estimators=500,
        max_depth=12,
        min_samples_leaf=2,
        max_features='sqrt',
        n_jobs=-1,
        random_state=SEED
    ),
    'ExtraTrees': lambda: ExtraTreesRegressor(
        n_estimators=500,
        max_depth=12,
        min_samples_leaf=2,
        max_features='sqrt',
        n_jobs=-1,
        random_state=SEED
    ),
    'GradientBoosting': lambda: GradientBoostingRegressor(
        n_estimators=1000,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.8,
        max_features='sqrt',
        random_state=SEED
    ),
    'HistGradientBoosting': lambda: HistGradientBoostingRegressor(
        max_iter=1000,
        learning_rate=0.05,
        max_leaf_nodes=31,
        early_stopping='auto',
        random_state=SEED
    ),
    'XGBoost': lambda: XGBRegressor(
        n_estimators=1000,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        tree_method='auto',   # change to 'gpu_hist' if GPU-compatible wheel and GPU available
        random_state=SEED,
        verbosity=0,
        n_jobs=-1
    ),
    'LightGBM': lambda: LGBMRegressor(
        n_estimators=1000,
        learning_rate=0.05,
        num_leaves=64,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=SEED,
        n_jobs=-1,
        verbose=-1
    ),
    'CatBoost': lambda: CatBoostRegressor(
        iterations=1000,
        learning_rate=0.05,
        depth=6,
        early_stopping_rounds=50,
        random_seed=SEED,
        verbose=0
    ),
}


for name, model_cls in models_to_compare.items():
    try:
        score = model(tc, test, model_cls, 'Tc', submission=False)
        print(f'{name:<20}: MAE = {score:.5f}')
    except Exception as e:
        print(f'{name:<20}: Failed with error: {e}')

RandomForest        : MAE = 0.03529
ExtraTrees          : MAE = 0.03622
GradientBoosting    : MAE = 0.03475
HistGradientBoosting: MAE = 0.03940
XGBoost             : MAE = 0.03547
LightGBM            : MAE = 0.03939
CatBoost            : MAE = 0.03472


In [27]:
models_to_compare = {
    'RandomForest': lambda: RandomForestRegressor(
        n_estimators=500,
        max_depth=12,
        min_samples_leaf=2,
        max_features='sqrt',
        n_jobs=-1,
        random_state=SEED
    ),
    'ExtraTrees': lambda: ExtraTreesRegressor(
        n_estimators=500,
        max_depth=12,
        min_samples_leaf=2,
        max_features='sqrt',
        n_jobs=-1,
        random_state=SEED
    ),
    'GradientBoosting': lambda: GradientBoostingRegressor(
        n_estimators=1000,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.8,
        max_features='sqrt',
        random_state=SEED
    ),
    'HistGradientBoosting': lambda: HistGradientBoostingRegressor(
        max_iter=1000,
        learning_rate=0.05,
        max_leaf_nodes=31,
        early_stopping='auto',
        random_state=SEED
    ),
    'XGBoost': lambda: XGBRegressor(
        n_estimators=1000,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        tree_method='auto',   # change to 'gpu_hist' if GPU-compatible wheel and GPU available
        random_state=SEED,
        verbosity=0,
        n_jobs=-1
    ),
    'LightGBM': lambda: LGBMRegressor(
        n_estimators=1000,
        learning_rate=0.05,
        num_leaves=64,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=SEED,
        n_jobs=-1,
        verbose=-1
    ),
    'CatBoost': lambda: CatBoostRegressor(
        iterations=1000,
        learning_rate=0.05,
        depth=6,
        early_stopping_rounds=50,
        random_seed=SEED,
        verbose=0
    ),
}

for name, model_cls in models_to_compare.items():
    try:
        score = model(density, test, model_cls, 'Density', submission=False)
        print(f'{name:<20}: MAE = {score:.5f}')
    except Exception as e:
        print(f'{name:<20}: Failed with error: {e}')

RandomForest        : MAE = 0.05104
ExtraTrees          : MAE = 0.05732
GradientBoosting    : MAE = 0.04161
HistGradientBoosting: MAE = 0.04383
XGBoost             : MAE = 0.04137
LightGBM            : MAE = 0.04292
CatBoost            : MAE = 0.03981


In [28]:
models_to_compare = {
    'RandomForest': lambda: RandomForestRegressor(
        n_estimators=500,
        max_depth=12,
        min_samples_leaf=2,
        max_features='sqrt',
        n_jobs=-1,
        random_state=SEED
    ),
    'ExtraTrees': lambda: ExtraTreesRegressor(
        n_estimators=500,
        max_depth=12,
        min_samples_leaf=2,
        max_features='sqrt',
        n_jobs=-1,
        random_state=SEED
    ),
    'GradientBoosting': lambda: GradientBoostingRegressor(
        n_estimators=1000,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.8,
        max_features='sqrt',
        random_state=SEED
    ),
    'HistGradientBoosting': lambda: HistGradientBoostingRegressor(
        max_iter=1000,
        learning_rate=0.05,
        max_leaf_nodes=31,
        early_stopping='auto',
        random_state=SEED
    ),
    'XGBoost': lambda: XGBRegressor(
        n_estimators=1000,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        tree_method='auto',   # change to 'gpu_hist' if GPU-compatible wheel and GPU available
        random_state=SEED,
        verbosity=0,
        n_jobs=-1
    ),
    'LightGBM': lambda: LGBMRegressor(
        n_estimators=1000,
        learning_rate=0.05,
        num_leaves=64,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=SEED,
        n_jobs=-1,
        verbose=-1
    ),
    'CatBoost': lambda: CatBoostRegressor(
        iterations=1000,
        learning_rate=0.05,
        depth=6,
        early_stopping_rounds=50,
        random_seed=SEED,
        verbose=0
    ),
}


for name, model_cls in models_to_compare.items():
    try:
        score = model(rg, test, model_cls, 'Rg', submission=False)
        print(f'{name:<20}: MAE = {score:.5f}')
    except Exception as e:
        print(f'{name:<20}: Failed with error: {e}')

RandomForest        : MAE = 1.88553
ExtraTrees          : MAE = 2.04705
GradientBoosting    : MAE = 1.95449
HistGradientBoosting: MAE = 2.03189
XGBoost             : MAE = 1.76785
LightGBM            : MAE = 2.00985
CatBoost            : MAE = 1.83598


# Final Model For Submission

In [29]:
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor

sub = {
    'id': ID,
    'Tg': model(tg, test, lambda: CatBoostRegressor(verbose=0), 'Tg', submission=True),
    'FFV': model(ffv, test, lambda: CatBoostRegressor(verbose=0), 'FFV', submission=True),
    'Tc': model(tc, test, lambda: CatBoostRegressor(verbose=0), 'Tc', submission=True),
    'Density': model(density, test, lambda: CatBoostRegressor(verbose=0), 'Density', submission=True),
    'Rg': model(rg, test, RandomForestRegressor, 'Rg', submission=True)
}

In [30]:
sub["Tg"] = scaler.inverse_transform(np.array(sub["Tg"]).reshape(-1, 1))

In [31]:
sub["Tg"] = sub["Tg"].reshape(-1)

In [32]:
submission=pd.DataFrame(sub)

In [33]:
submission

,id,Tg,FFV,Tc,Density,Rg
0,1109053969,171.132528,0.375060,0.210824,1.153362,20.750384
1,1422188626,207.932944,0.379103,0.236521,1.083875,21.031708
2,2032016830,114.399049,0.354266,0.249140,1.083284,20.097919


In [34]:
submission.to_csv('submission.csv',index=False)